In [2]:
import os
import sys
from pathlib import Path
from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifWriter
from Xerus.utils.cifutils import make_combinations
from Xerus.settings.settings import MP_API_KEY
import os
import pandas as pd
from typing import Tuple, List
dump_folder = "mp_dump/"

def make_cifs(data: pd.DataFrame, symprec: float = 1e-2, folder_path: os.PathLike = dump_folder) -> None:
    """
    Write CIFs from data queried from Materials Project into a folder
    Parameters
    ----------
    data : A pandas dataframe containing the material id, formula and structure information
    symprec : Spglib symprec tolerance to find spacegroups (read pymatgen documentation for more detail)
    folder_path : Folder to save.
    Returns
    -------
    None
    """
    if not os.path.isdir(folder_path):
        os.mkdir(folder_path)
    for mid, name, struc in zip(data['material_id'], data['pretty_formula'], data.structure):
        writer = CifWriter(struc, symprec=symprec)
        filename = folder_path + os.sep + name + "_" + "MP_" + mid + ".cif"

        if name != 'O2':
            writer.write_file(filename)
            print("Writing {}".format(filename))

'''

Parameters
----------
inc_eles : Elements to query (inclusive) eg:["Ho", "B"]
exc_eles : Elements NOT to query (exclusive)
max_num_elem : Maximum number of elements
api_key : API-Key
combination : boolean. To combine or not the list of elements into a all possible system-types.
min_hull : Minimum e_above_hull to consider when downloading CIFS
write : To save the cifs into a folder or not
folder_path : If write = True, which folder to save.

Returns
-------
Returns a DataFrame with the queried data information with data is available for elements combination.
'''

testa = ["Ho"]
testb = ["B"]
testc = ["Ho", "B"]
inc_eles: List[str] = testa
exc_eles: List = []
min_hull: float = 1e-4
folder_path: os.PathLike = dump_folder
api_key: str = "igxn6S3EIb665F8gN"
write: bool =True
combination: bool = False
min_hull: float = 1e-4
max_num_elem = len(inc_eles)

a = MPRester(api_key)

qp = {"elements": {"$all": inc_eles, "$nin": exc_eles}, "nelements": {"$lte": max_num_elem}}
data = a.query(qp, ['pretty_formula', 'structure', 'theoretical', 'material_id', 'e_above_hull'])
if len(data) > 0:
    data = pd.DataFrame.from_dict(data)
    data = data[~data.theoretical]
    data = data[data.e_above_hull <= min_hull]
    data.reset_index(drop=True, inplace=True)

if len(data) == 0:
    print("no data")
if write:
    make_cifs(data, folder_path=folder_path)
data

GSAS-II binary directory: /home/wilson/Documents/Xerus/Xerus/GSASII/bindist
7 values read from config file /home/wilson/Documents/Xerus/Xerus/GSASII/config.py
Writing mp_dump//Ho_MP_mp-10659.cif


,pretty_formula,structure,theoretical,material_id,e_above_hull
0,Ho,"[[0. 0. 0.] Ho, [19.82845505 0. 2.37...",False,mp-10659,0.0


In [10]:
from mp_api.client import MPRester
# def querymp(inc_eles: List[str], max_num_elem:int = 3, min_hull: float = 1e-4,
#             folder_path: os.PathLike = dump_folder) -> pd.DataFrame:

api_key: str = MP_API_KEY
write: bool =True
testa = ["Ho"]
testb = ["B"]
testc = ["Ho-B"]
inc_eles: List[str] = testc
min_hull: float = 1e-4
properties = ['formula_pretty',  'material_id',  'structure',  'energy_above_hull',  'theoretical']
cifs = ['pretty_formula', 'material_id', 'structure', 'e_above_hull', 'theoretical', 'fields_not_requested']
min_hull = 0.1

with MPRester(api_key) as mpr:    
    datas = mpr.summary.search( chemsys=inc_eles,
                                fields = properties, 
                                theoretical = False, 
                                energy_above_hull = ( 0, min_hull))
    datadf = [data.dict() for data in datas]
    datadf = pd.DataFrame(datadf)
    print(datadf)
    # datadf = datadf.drop(columns = ['fields_not_requested'])
    datadf = datadf.set_axis(cifs, axis='columns')
    
if len(datadf) == 0:
    print("no data")
if write:
    make_cifs(datadf, folder_path=folder_path)
datadf

Retrieving SummaryDoc documents:   0%|          | 0/4 [00:00<?, ?it/s]

  formula_pretty material_id  \
0          HoB12  mp-1104585   
1           HoB2     mp-2267   
2           HoB4   mp-569281   
3           HoB6   mp-571491   

                                           structure  energy_above_hull  \
0  [[0. 0. 0.] Ho, [5.00945472 5.00945472 3.74457...           0.000000   
1  [[0. 0. 0.] Ho, [1.643563   0.94891356 1.91523...           0.000000   
2  [[2.25102619 1.29206681 0.        ] Ho, [4.835...           0.000000   
3  [[0. 0. 0.] Ho, [0.81314035 2.0452965  2.04529...           0.050043   

   theoretical                               fields_not_requested  
0        False  [builder_meta, nsites, elements, nelements, co...  
1        False  [builder_meta, nsites, elements, nelements, co...  
2        False  [builder_meta, nsites, elements, nelements, co...  
3        False  [builder_meta, nsites, elements, nelements, co...  
Writing mp_dump//HoB12_MP_mp-1104585.cif
Writing mp_dump//HoB2_MP_mp-2267.cif
Writing mp_dump//HoB4_MP_mp-569281.cif
Writin

,pretty_formula,material_id,structure,e_above_hull,theoretical,fields_not_requested
0,HoB12,mp-1104585,"[[0. 0. 0.] Ho, [5.00945472 5.00945472 3.74457...",0.000000,False,"[builder_meta, nsites, elements, nelements, co..."
1,HoB2,mp-2267,"[[0. 0. 0.] Ho, [1.643563 0.94891356 1.91523...",0.000000,False,"[builder_meta, nsites, elements, nelements, co..."
2,HoB4,mp-569281,"[[2.25102619 1.29206681 0. ] Ho, [4.835...",0.000000,False,"[builder_meta, nsites, elements, nelements, co..."
3,HoB6,mp-571491,"[[0. 0. 0.] Ho, [0.81314035 2.0452965 2.04529...",0.050043,False,"[builder_meta, nsites, elements, nelements, co..."


In [ ]:
mongosh
show databases
use <databasename>
db.dropDatabase()

===

properties = ['formula_pretty',  'structure',  'theoretical',  'material_id',  'energy_above_hull']

Idea is: e_bove_hull <= min_hull

min_hull should be set to 0.1 in the query mp

no theoretical structure (theoritcal = False)